<h1 style="font-size:50px;" align="center">Inferencia Bayesiana usando el Motor de Inferencia MIB</h1>

In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd

import mib_v1_2 as mb

('A', 0) 0.3
('A', 1) 0.7
{('A', 0): {('B', 0): 0.2, ('B', 1): 0.8}, ('A', 1): {('B', 0): 0.3, ('B', 1): 0.7}}
computing marginal P(B)
('B', 0) 0.27
('B', 1) 0.7299999999999999
Printing:  P(B)
('B', 0):0.27  ('B', 1):0.7299999999999999  

None


### Variables

* Se declaran las siguientes variables:
    - A:= {0,1}
    - B:= {0,1}
    - C:= {0,1,2}
    - D:= {0,1,2}

In [2]:
A=mb.Var('A')
A.set_values([0,1])
B=mb.Var('B',[0,1])
C=mb.Var('C',[0,1,2])
D=mb.Var('D',[0,1,2])

In [3]:
print(f'Cardinalidad de A={A.card}, Cardinalidad de C={C.card}')

Cardinalidad de A=2, Cardinalidad de C=3


In [4]:
A.print_var()

('A', 2, [0, 1])

In [5]:
print(A.print_var())
print(B.print_var())
print(C.print_var())
print(D.print_var())

('A', 2, [0, 1])
('B', 2, [0, 1])
('C', 3, [0, 1, 2])
('D', 3, [0, 1, 2])


## Modelo de descomposición exacta

* Deseamos construir el siguiente modelo:
    - $P(ABC) = P(A)P(B|A)P(C|AB)$
    
$\;\;\;\;\;\;\;\;\;\;\;\;$<img src="./P(ABC).jpg" width=200/>

### Identificación de parámetros

* Definimos sus distribuciones (de manera arbitraria)

<table width="90%">
    <tr>
        <td valign="top">
            <table width="50%">
              <tr>
                <th> $A$</th>
                <th><p style="text-align:center;"> $P(A)$</p></th>
              </tr>
              <tr>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
              </tr>
            </table>
        </td>
        <td valign="top">
            <table width="60%">
              <tr>
                <td style="height:1px;"> &nbsp;</td>
                <td colspan="2"><p style="text-align:center;">$P$($B$|$A$)</p></td>
              </tr>
              <tr>
                <td>$A$</td>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
                <td><p style="text-align:center;">$0.8$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
              </tr>
            </table>        
        </td>
        <td valign="top">
            <table width="90%">
              <tr>
                <td style="height:1px;"> &nbsp;</td>
                <td colspan="2"><p style="text-align:center;">$P$($C$|$AB$)</p></td>
              </tr>
              <tr>
                <td>$AB$</td>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$2$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$00$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
                <td><p style="text-align:center;">$0.8$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$01$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
                <td><p style="text-align:center;">$0.5$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$10$</p></td>
                <td><p style="text-align:center;">$0.4$</p></td>
                <td><p style="text-align:center;">$0.5$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$11$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
              </tr>
            </table>        
        </td>
    </tr>
</table>


### Construcción de las distribuciones

In [6]:
tabla = {0: {(0,0): 0.1, (0,1): 0.3, (1,0): 0.4, (1,1): 0.1},
        1: {(0,0): 0.8, (0,1): 0.5, (1,0): 0.5, (1,1): 0.7},
        2: {(0,0): 0.1, (0,1): 0.2, (1,0): 0.1, (1,1): 0.2}}
tabla.values()

dict_values([{(0, 0): 0.1, (0, 1): 0.3, (1, 0): 0.4, (1, 1): 0.1}, {(0, 0): 0.8, (0, 1): 0.5, (1, 0): 0.5, (1, 1): 0.7}, {(0, 0): 0.1, (0, 1): 0.2, (1, 0): 0.1, (1, 1): 0.2}])

In [7]:
dA={0:0.3,1:0.7}
PA=mb.Distrib(name='P(A)',variable=[A],tabla=dA)

dB_A={0:{0:.2,1:.8},1:{0:.3,1:.7}}
PB_A=mb.DistribCond(name='P(B|A)',var=B,indep=[A],tabla=dB_A)

tabla = {(0,0): (0.1,0.8,0.1),
        (0,1): (0.3,0.5,0.2),
        (1,0): (0.4,0.5,0.1),
        (1,1): (0.1,0.7,0.2)}
PC_AB=mb.DistribCond('P(C|AB)',C,[A,B],tabla)

PA._print()
PC_AB._print()

('A', 0) 0.3
('A', 1) 0.7
Printing:  P(A)
('A', 0):0.3  ('A', 1):0.7  

Printing:  P(C|AB)
('A', 'B', 0, 0):{('C', 0): 0.1, ('C', 1): 0.8, ('C', 2): 0.1}  ('A', 'B', 0, 1):{('C', 0): 0.3, ('C', 1): 0.5, ('C', 2): 0.2}  ('A', 'B', 1, 0):{('C', 0): 0.4, ('C', 1): 0.5, ('C', 2): 0.1}  ('A', 'B', 1, 1):{('C', 0): 0.1, ('C', 1): 0.7, ('C', 2): 0.2}  



In [8]:
PA.to_Frame()

Creating DataFrame for:  P(A)


P(A)
(A, 0)   0.3
(A, 1)   0.7

### Construcción de la distribución conjunta con base en la descomposición

In [9]:
PABC=mb.JointDistrib(name='P(ABC)',variables=[A,B,C],descomp=[PA,PB_A,PC_AB])
PABC.to_Frame()

Computing marginal joint: P(A,B,C)
('A', 'B', 'C', 0, 0, 0) 0.006000000000000001
('A', 'B', 'C', 0, 0, 1) 0.04800000000000001
('A', 'B', 'C', 0, 0, 2) 0.006000000000000001
('A', 'B', 'C', 0, 1, 0) 0.072
('A', 'B', 'C', 0, 1, 1) 0.12
('A', 'B', 'C', 0, 1, 2) 0.04800000000000001
('A', 'B', 'C', 1, 0, 0) 0.08399999999999999
('A', 'B', 'C', 1, 0, 1) 0.105
('A', 'B', 'C', 1, 0, 2) 0.020999999999999998
('A', 'B', 'C', 1, 1, 0) 0.048999999999999995
('A', 'B', 'C', 1, 1, 1) 0.3429999999999999
('A', 'B', 'C', 1, 1, 2) 0.09799999999999999


P(ABC)
(A, B, C, 0, 0, 0)   0.006
(A, B, C, 0, 0, 1)   0.048
(A, B, C, 0, 0, 2)   0.006
(A, B, C, 0, 1, 0)   0.072
(A, B, C, 0, 1, 1)   0.120
(A, B, C, 0, 1, 2)   0.048
(A, B, C, 1, 0, 0)   0.084
(A, B, C, 1, 0, 1)   0.105
(A, B, C, 1, 0, 2)   0.021
(A, B, C, 1, 1, 0)   0.049
(A, B, C, 1, 1, 1)   0.343
(A, B, C, 1, 1, 2)   0.098

In [10]:
PB_A.get_P()

{('A', 0): {('B', 0): 0.2, ('B', 1): 0.8},
 ('A', 1): {('B', 0): 0.3, ('B', 1): 0.7}}

### Construcción de la Pregunta

* Creamos la estructura de pregunta

In [11]:
Q_ABC=mb.Question(joint=PABC)

### Formulación de la consulta (query)

* Formulamos una consulta (query): $P(B)$

In [12]:
PB=Q_ABC.query(searched=[B])
PB._print()

computing marginal P(B)
('B', 0) 0.27
('B', 1) 0.7299999999999999
Printing:  P(B)
('B', 0):0.27  ('B', 1):0.7299999999999999  



#### Ejercicio resuelto:
* Comprueba por tu cuenta (manualmente) que $P(B)$ es correcta 
    - Necesitas calcular $$P(B)=\sum_{AC}P(ABC)$$
    - donde $$P(ABC)=P(A)P(B|A)P(C|AB)$$

* Algoritmo:
1. &emsp;Comenzamos escribiendo $P(B=0)$ como la suma sobre todos los valores posibles de $A$ y $C$ en la conjunta:<br>
&emsp;$P(B=0)=P(A=0)P(B=0|A=0)P(C=0|A=0 \wedge B=0)+$<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$P(A=1)P(B=0|A=1)P(C=0|A=1 \wedge B=0)+$<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$P(A=0)P(B=0|A=0)P(C=1|A=0 \wedge B=0)+$<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$P(A=1)P(B=0|A=1)P(C=1|A=1 \wedge B=0)+$<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$P(A=0)P(B=0|A=0)P(C=2|A=0 \wedge B=0)+$<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$P(A=1)P(B=0|A=1)P(C=2|A=1 \wedge B=0)$
2. &emsp;Simplificamos el cálculo factorizando los términos en $A$ y $B$:<br>
&emsp;$P(B=0)=P(A=0)P(B=0|A=0)\displaystyle \sum_C P(C|A=0 \wedge B=0)+$<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$P(A=1)P(B=0|A=1)\sum_C P(C|A=1 \wedge B=0)$<br>
3. &emsp;Como $\displaystyle \sum_C P(C|A=0 \wedge B=0)=1$:<br>
&emsp;$P(B=0)=P(A=0)P(B=0|A=0)+P(A=1)P(B=0|A=1)$<br>
4. &emsp;El algoritmo entonces:
    - Multiplica los valores de  $P(B|A)\times P(A)$ para la llave ('A',0) y lo suma al mismo producto para la llave ('A',1).
    - Repite el proceso para $B=1$. 

In [13]:
pa=PA.get_P()
pb_a=PB_A.get_P()

print(pa.items())
print(pb_a.items())

dict_items([(('A', 0), 0.3), (('A', 1), 0.7)])
dict_items([(('A', 0), {('B', 0): 0.2, ('B', 1): 0.8}), (('A', 1), {('B', 0): 0.3, ('B', 1): 0.7})])


In [14]:
#P(A=0)
print(PA.get_P(('A',0)))

#P(B=0|A=0)
print(PB_A.get_P([('A',0)])[0][('B',0)])

0.3
0.2


In [15]:
P_B=[]
for b in range(B.card):   #b={0,1}
    suma=0
    for i,ka in enumerate(pb_a.keys()): #para ambas llaves ka en P(B|ka)
        kb=list(pb_a[ka].keys())[b] #obtiene la llave kb para P(kb|ka)
        print(f'Obtenemos P({kb[0]}={kb[1]}|{ka[0]}={ka[1]})')
        a=list(pa.keys())[i]        #obtiene la llave a de P(A=i)
        print(f'Obtenemos P({a[0]}={a[1]})')
        pb=pb_a[ka][kb]*pa[a]       #calcula P(kb|ka)xP(a)
        print(f'Calculamos P({kb[0]}={kb[1]}|{ka[0]}={ka[1]})xP({a[0]}={a[1]})')
        suma+=pb                    #calcula SUM[P(kb|ka)xP(a)]
    print(f'valor obtenido luego de la suma: {suma}')
    P_B.append(suma)
print(P_B)

Obtenemos P(B=0|A=0)
Obtenemos P(A=0)
Calculamos P(B=0|A=0)xP(A=0)
Obtenemos P(B=0|A=1)
Obtenemos P(A=1)
Calculamos P(B=0|A=1)xP(A=1)
valor obtenido luego de la suma: 0.27
Obtenemos P(B=1|A=0)
Obtenemos P(A=0)
Calculamos P(B=1|A=0)xP(A=0)
Obtenemos P(B=1|A=1)
Obtenemos P(A=1)
Calculamos P(B=1|A=1)xP(A=1)
valor obtenido luego de la suma: 0.73
[0.27, 0.73]


### Marginal de $C$

In [16]:
PC=Q_ABC.query([C])
PC._print()

computing marginal P(C)
('C', 0) 0.21099999999999997
('C', 1) 0.6159999999999999
('C', 2) 0.173
Printing:  P(C)
('C', 0):0.21099999999999997  ('C', 1):0.6159999999999999  ('C', 2):0.173  



#### Ejercicio:
* Comprueba por tu cuenta (manualmente) que $P(C)$ es correcta 
    - Necesitas calcular $$P(C)=\sum_{AB}P(ABC)$$
    - donde $$P(ABC)=P(A)P(B|A)P(C|AB)$$

* Algoritmo:
1. &emsp;Comienza analizando $P(C=0)$:<br>
&emsp;$P(C=0)=P(A=0)P(B=0|A=0)P(C=0|A=0 \wedge B=0)+$<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$P(A=1)P(B=0|A=1)P(C=0|A=1 \wedge B=0)+$<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$P(A=0)P(B=1|A=0)P(C=0|A=0 \wedge B=1)+$<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$P(A=1)P(B=1|A=1)P(C=0|A=1 \wedge B=1)$<br>
2. &emsp;Reescribe $P(C=0)$ para simplificar la expresión.
3. &emsp;Escribe el algoritmo.

#### TU RESPUESTA AQUI
2. &emsp;$P(C=0)=P(A=0)\sum_B P(B|A=0)P(C=0|A=0 \wedge B)+P(A=1)\sum_B P(B|A=1)P(C=0|A=1 \wedge B)$
3. &emsp;El algoritmo entonces:
    - Multiplica los valores de  $P(B|A=0)\times P(C=0|A=0B)$ y los suma, para todos los valores de $B$.
    - Multiplica $P(A=0)$ con la suma anterior.
    - Repite el proceso para $A=1$.
    - Suma estos valores
    - Repite para los demás valores de $C$

In [17]:
pc_ab=PC_AB.get_P()

print(pa.items())
print(pb_a.items())
print(pc_ab.items())

dict_items([(('A', 0), 0.3), (('A', 1), 0.7)])
dict_items([(('A', 0), {('B', 0): 0.2, ('B', 1): 0.8}), (('A', 1), {('B', 0): 0.3, ('B', 1): 0.7})])
dict_items([(('A', 'B', 0, 0), {('C', 0): 0.1, ('C', 1): 0.8, ('C', 2): 0.1}), (('A', 'B', 0, 1), {('C', 0): 0.3, ('C', 1): 0.5, ('C', 2): 0.2}), (('A', 'B', 1, 0), {('C', 0): 0.4, ('C', 1): 0.5, ('C', 2): 0.1}), (('A', 'B', 1, 1), {('C', 0): 0.1, ('C', 1): 0.7, ('C', 2): 0.2})])


In [18]:
P_C=[]
for c in range(C.card): #para todos los valores de C
    psum=0
    for i,ka in enumerate(pb_a.keys()): #para ambas llaves en P(B|A)
        suma=0
        for b in range(B.card): #para todos los valores de B
            kb=list(pb_a[ka].keys())[b]
            llave = list(pc_ab.keys())[b+2*i]
            prob = list(pc_ab[llave].keys())[c]
            print(ka, kb,end=' x ')
            print(llave, prob)
            """ TU CODIGO AQUI """
            p1=pb_a[ka][kb]*pc_ab[llave][prob]
            suma+=p1
        p_c = pa[ka]*suma
        psum += p_c
    P_C.append(psum)
print(P_C)
            

('A', 0) ('B', 0) x ('A', 'B', 0, 0) ('C', 0)
('A', 0) ('B', 1) x ('A', 'B', 0, 1) ('C', 0)
('A', 1) ('B', 0) x ('A', 'B', 1, 0) ('C', 0)
('A', 1) ('B', 1) x ('A', 'B', 1, 1) ('C', 0)
('A', 0) ('B', 0) x ('A', 'B', 0, 0) ('C', 1)
('A', 0) ('B', 1) x ('A', 'B', 0, 1) ('C', 1)
('A', 1) ('B', 0) x ('A', 'B', 1, 0) ('C', 1)
('A', 1) ('B', 1) x ('A', 'B', 1, 1) ('C', 1)
('A', 0) ('B', 0) x ('A', 'B', 0, 0) ('C', 2)
('A', 0) ('B', 1) x ('A', 'B', 0, 1) ('C', 2)
('A', 1) ('B', 0) x ('A', 'B', 1, 0) ('C', 2)
('A', 1) ('B', 1) x ('A', 'B', 1, 1) ('C', 2)
[0.21099999999999997, 0.6159999999999999, 0.173]


### Condicional de $C|A$

In [19]:
# P(C|A)
PC_A=Q_ABC.query(searched=[C],known=[A])
PC_A._print()

computing conditional: P(C|A)
Printing:  P(C|A)
('A', 0):{('C', 0): 0.25999999999999995, ('C', 1): 0.5599999999999999, ('C', 2): 0.18}  ('A', 1):{('C', 0): 0.19, ('C', 1): 0.64, ('C', 2): 0.17000000000000004}  



### Conjunta de $A \wedge C$

In [20]:
# P(AC)
PAC=Q_ABC.query([A,C])
PAC._print()

Computing marginal joint: P(A,C)
('A', 'C', 0, 0) 0.078
('A', 'C', 0, 1) 0.168
('A', 'C', 0, 2) 0.054000000000000006
('A', 'C', 1, 0) 0.13299999999999998
('A', 'C', 1, 1) 0.4479999999999999
('A', 'C', 1, 2) 0.119
Printing:  P(A,C)
('A', 'C', 0, 0):0.078  ('A', 'C', 0, 1):0.168  ('A', 'C', 0, 2):0.054000000000000006  ('A', 'C', 1, 0):0.13299999999999998  ('A', 'C', 1, 1):0.4479999999999999  ('A', 'C', 1, 2):0.119  



## Modelo tipo _Bayes Ingenuo_

### Descomposición
* $P(ABCD) = P(A)P(B|A)P(C|A)P(D|A)$
    
$\;\;\;\;\;\;\;\;\;\;\;\;$<img src="./Bayes_Ingenuo.jpg" width=200/>

In [22]:
dA={0:0.3,1:0.7}
PA=mb.Distrib('P(A)',[A],dA)

dB_A={0:{0:.2,1:.8},1:{0:.3,1:.7}}
PB_A=mb.DistribCond('P(B|A)',var=B,indep=[A],tabla=dB_A)

dC_A = {0: {0:.1, 1: .8, 2:.1}, 1:{0:.3, 1:.5, 2:.2}}
PC_A=mb.DistribCond('P(C|A)',C,[A],dC_A)

dD_A={0:{0:0.333,1:0.333,2:0.333},1:{0:0.8,1:0.15,2:0.05},}
PD_A=mb.DistribCond('P(D|A)',D,[A],dD_A)

('A', 0) 0.3
('A', 1) 0.7


### Conjunta y Pregunta

In [23]:
PABCD=mb.JointDistrib(name='P(ABCD)',variables=[A,B,C,D],descomp=[PA,PB_A,PC_A,PD_A])
Q_ABCD=mb.Question(PABCD)

PABCD._print()

P(B|A) : {('A', 0): {('B', 0): 0.2, ('B', 1): 0.8}, ('A', 1): {('B', 0): 0.3, ('B', 1): 0.7}}
P(C|A) : {('A', 0): {('C', 0): 0.1, ('C', 1): 0.8, ('C', 2): 0.1}, ('A', 1): {('C', 0): 0.3, ('C', 1): 0.5, ('C', 2): 0.2}}
P(D|A) : {('A', 0): {('D', 0): 0.333, ('D', 1): 0.333, ('D', 2): 0.333}, ('A', 1): {('D', 0): 0.8, ('D', 1): 0.15, ('D', 2): 0.05}}
P(A) : {('A', 0): 0.3, ('A', 1): 0.7}


### Marginales

In [24]:
# P(A), P(B), P(C) y P(D)
PAc=Q_ABCD.query([A])
PBc=Q_ABCD.query([B])
PCc=Q_ABCD.query([C])
PDc=Q_ABCD.query([D])
PAc._print()
PBc._print()
PCc._print()
PDc._print()

computing marginal P(A)
('A', 0) 0.2997
('A', 1) 0.7
computing marginal P(B)
('B', 0) 0.26994
('B', 1) 0.72976
computing marginal P(C)
('C', 0) 0.23997
('C', 1) 0.5897600000000001
('C', 2) 0.16996999999999998
computing marginal P(D)
('D', 0) 0.6598999999999999
('D', 1) 0.2049
('D', 2) 0.1349
Printing:  P(A)
('A', 0):0.2997  ('A', 1):0.7  

Printing:  P(B)
('B', 0):0.26994  ('B', 1):0.72976  

Printing:  P(C)
('C', 0):0.23997  ('C', 1):0.5897600000000001  ('C', 2):0.16996999999999998  

Printing:  P(D)
('D', 0):0.6598999999999999  ('D', 1):0.2049  ('D', 2):0.1349  



### Condicionales directas

$\;\;\;\;\;\;\;\;\;\;\;\;$<img src="./Directo.jpg" width=200/>

In [25]:
# P(B|A), P(C|A) y P(D|A)
PB_Ac=Q_ABCD.query([B],[A])
PC_Ac=Q_ABCD.query([C],[A])
PD_Ac=Q_ABCD.query([D],[A])
PB_Ac._print()
PC_Ac._print()
PD_Ac._print()

computing conditional: P(B|A)
computing conditional: P(C|A)
computing conditional: P(D|A)
Printing:  P(B|A)
('A', 0):{('B', 0): 0.2, ('B', 1): 0.8}  ('A', 1):{('B', 0): 0.3, ('B', 1): 0.7}  

Printing:  P(C|A)
('A', 0):{('C', 0): 0.1, ('C', 1): 0.8, ('C', 2): 0.1}  ('A', 1):{('C', 0): 0.3, ('C', 1): 0.5, ('C', 2): 0.19999999999999998}  

Printing:  P(D|A)
('A', 0):{('D', 0): 0.3333333333333333, ('D', 1): 0.3333333333333333, ('D', 2): 0.3333333333333333}  ('A', 1):{('D', 0): 0.7999999999999999, ('D', 1): 0.15, ('D', 2): 0.049999999999999996}  



### Condicionales inversas

$\;\;\;\;\;\;\;\;\;\;\;\;$<img src="./Inverso.jpg" width=200/>

In [27]:
# P(A|B), P(A|C) y P(A|D)
PA_Bc=Q_ABCD.query([A],[B])
PA_Cc=Q_ABCD.query([A],[C])
PA_Dc=Q_ABCD.query([A],[D])
PA_Bc._print()
PA_Cc._print()
PA_Dc._print()

computing conditional: P(A|B)
computing conditional: P(A|C)
computing conditional: P(A|D)
Printing:  P(A|B)
('B', 0):{('A', 0): 0.22204934429873308, ('A', 1): 0.7779506557012669}  ('B', 1):{('A', 0): 0.3285463714097786, ('A', 1): 0.6714536285902214}  

Printing:  P(A|C)
('C', 0):{('A', 0): 0.12489061132641582, ('A', 1): 0.8751093886735842}  ('C', 1):{('A', 0): 0.4065382528486164, ('A', 1): 0.5934617471513836}  ('C', 2):{('A', 0): 0.1763252338647997, ('A', 1): 0.8236747661352003}  

Printing:  P(A|D)
('D', 0):{('A', 0): 0.15138657372329142, ('A', 1): 0.8486134262767085}  ('D', 1):{('A', 0): 0.4875549048316252, ('A', 1): 0.5124450951683748}  ('D', 2):{('A', 0): 0.7405485544848036, ('A', 1): 0.25945144551519644}  



### Condicionales "transversales"

$\;\;\;\;\;\;\;\;\;\;\;\;$<img src="./Transversal.jpg" width=200/>

In [28]:
#P(B|C), P(C|B), P(D|BC) , P(BC|D)
PB_Cc=Q_ABCD.query([B],[C])
PB_Cc._print()

PC_Bc=Q_ABCD.query([C],[B])
PC_Bc._print()

PD_BCc=Q_ABCD.query([D],[B,C])
df1=PD_BCc.to_Frame()

PBC_Dc=Q_ABCD.query([B,C],[D])
df2=PBC_Dc.to_Frame()


computing conditional: P(B|C)
Printing:  P(B|C)
('C', 0):{('B', 0): 0.28751093886735846, ('B', 1): 0.7124890611326415}  ('C', 1):{('B', 0): 0.2593461747151383, ('B', 1): 0.7406538252848616}  ('C', 2):{('B', 0): 0.28236747661352, ('B', 1): 0.7176325233864801}  

computing conditional: P(C|B)
Printing:  P(C|B)
('B', 0):{('C', 0): 0.25559013114025336, ('C', 1): 0.5666148032896199, ('C', 2): 0.17779506557012667}  ('B', 1):{('C', 0): 0.2342907257180443, ('C', 1): 0.5985639114229335, ('C', 2): 0.16714536285902215}  

computing conditional: P(D|B,C)
computing conditional of joint: P(B,C|D)


In [29]:
df1

(D, 0)    (D, 1)    (D, 2)
(B, C, 0, 0)  0.759457  0.165927  0.074615
(B, C, 0, 1)  0.653695  0.207477  0.138828
(B, C, 0, 2)  0.741718  0.172897  0.085386
(B, C, 1, 0)  0.734559  0.175709  0.089732
(B, C, 1, 1)  0.595081  0.230504  0.174415
(B, C, 1, 2)  0.708270  0.186037  0.105693

In [30]:
df2

(B, C, 0, 0) (B, C, 0, 1) (B, C, 0, 2) (B, C, 1, 0) (B, C, 1, 1)  \
(D, 0)     0.079403     0.151514     0.053945     0.190320     0.393902   
(D, 1)     0.055871     0.154876     0.040498     0.146618     0.491391   
(D, 2)     0.038162     0.157405     0.030378     0.113729     0.564759   

       (B, C, 1, 2)  
(D, 0)     0.130917  
(D, 1)     0.110747  
(D, 2)     0.095567

### Marginales conjuntas

In [31]:
# P(AB), P(AC) y P(AD)
PABc=Q_ABCD.query([A,B])
df=PABc.to_Frame()
df

Computing marginal joint: P(A,B)
('A', 'B', 0, 0) 0.05994000000000001
('A', 'B', 0, 1) 0.23976000000000003
('A', 'B', 1, 0) 0.21
('A', 'B', 1, 1) 0.48999999999999994
Creating DataFrame for:  P(A,B)


P(A,B)
(A, B, 0, 0)  0.05994
(A, B, 0, 1)  0.23976
(A, B, 1, 0)  0.21000
(A, B, 1, 1)  0.49000

### Distribución conjunta

In [32]:
# P(ABCD)
PABCDc=Q_ABCD.query([A,B,C,D])
df=PABCDc.to_Frame()
df

Computing marginal joint: P(A,B,C,D)
('A', 'B', 'C', 'D', 0, 0, 0, 0) 0.0019980000000000006
('A', 'B', 'C', 'D', 0, 0, 0, 1) 0.0019980000000000006
('A', 'B', 'C', 'D', 0, 0, 0, 2) 0.0019980000000000006
('A', 'B', 'C', 'D', 0, 0, 1, 0) 0.015984000000000005
('A', 'B', 'C', 'D', 0, 0, 1, 1) 0.015984000000000005
('A', 'B', 'C', 'D', 0, 0, 1, 2) 0.015984000000000005
('A', 'B', 'C', 'D', 0, 0, 2, 0) 0.0019980000000000006
('A', 'B', 'C', 'D', 0, 0, 2, 1) 0.0019980000000000006
('A', 'B', 'C', 'D', 0, 0, 2, 2) 0.0019980000000000006
('A', 'B', 'C', 'D', 0, 1, 0, 0) 0.007992000000000003
('A', 'B', 'C', 'D', 0, 1, 0, 1) 0.007992000000000003
('A', 'B', 'C', 'D', 0, 1, 0, 2) 0.007992000000000003
('A', 'B', 'C', 'D', 0, 1, 1, 0) 0.06393600000000002
('A', 'B', 'C', 'D', 0, 1, 1, 1) 0.06393600000000002
('A', 'B', 'C', 'D', 0, 1, 1, 2) 0.06393600000000002
('A', 'B', 'C', 'D', 0, 1, 2, 0) 0.007992000000000003
('A', 'B', 'C', 'D', 0, 1, 2, 1) 0.007992000000000003
('A', 'B', 'C', 'D', 0, 1, 2, 2) 0.0079920

P(A,B,C,D)
(A, B, C, D, 0, 0, 0, 0)    0.001998
(A, B, C, D, 0, 0, 0, 1)    0.001998
(A, B, C, D, 0, 0, 0, 2)    0.001998
(A, B, C, D, 0, 0, 1, 0)    0.015984
(A, B, C, D, 0, 0, 1, 1)    0.015984
(A, B, C, D, 0, 0, 1, 2)    0.015984
(A, B, C, D, 0, 0, 2, 0)    0.001998
(A, B, C, D, 0, 0, 2, 1)    0.001998
(A, B, C, D, 0, 0, 2, 2)    0.001998
(A, B, C, D, 0, 1, 0, 0)    0.007992
(A, B, C, D, 0, 1, 0, 1)    0.007992
(A, B, C, D, 0, 1, 0, 2)    0.007992
(A, B, C, D, 0, 1, 1, 0)    0.063936
(A, B, C, D, 0, 1, 1, 1)    0.063936
(A, B, C, D, 0, 1, 1, 2)    0.063936
(A, B, C, D, 0, 1, 2, 0)    0.007992
(A, B, C, D, 0, 1, 2, 1)    0.007992
(A, B, C, D, 0, 1, 2, 2)    0.007992
(A, B, C, D, 1, 0, 0, 0)    0.050400
(A, B, C, D, 1, 0, 0, 1)    0.009450
(A, B, C, D, 1, 0, 0, 2)    0.003150
(A, B, C, D, 1, 0, 1, 0)    0.084000
(A, B, C, D, 1, 0, 1, 1)    0.015750
(A, B, C, D, 1, 0, 1, 2)    0.005250
(A, B, C, D, 1, 0, 2, 0)    0.033600
(A, B, C, D, 1, 0, 2, 1)    0.006300
(A, B, C, D, 1, 0, 2, 2)    0.002100
(A, B, C, D, 1, 1, 0, 0)    0.117600
(A, B, C, D, 1, 1, 0, 1)    0.022050
(A, B, C, D, 1, 1, 0, 2)    0.007350
(A, B, C, D, 1, 1, 1, 0)    0.196000
(A, B, C, D, 1, 1, 1, 1)    0.036750
(A, B, C, D, 1, 1, 1, 2)    0.012250
(A, B, C, D, 1, 1, 2, 0)    0.078400
(A, B, C, D, 1, 1, 2, 1)    0.014700
(A, B, C, D, 1, 1, 2, 2)    0.004900

## SOLUCION TAREA FILTRO SPAM

### Principio de solución
* Tenemos las siguientes variables:
    - $Spam:=\{0,1\}$ con $\{0:.25, 1:.75\}$.
    - $W_i:=\{0,1\}$ con sus respectivas probabilidades dadas por la Tabla 1.
* Conjunta: 
    - $P(Spam, W^4_0)=P(Spam)\displaystyle \prod_i P(W_i|Spam)$.
* Consulta:
    - $P(W_i|Spam)$.

### Variables

#### Spam

In [35]:
#Spam
Spam=mb.Var('Spam')
Spam.set_values([0,1])
Spam.print_var()

('Spam', 2, [0, 1])

#### $W_i$

In [34]:
#Palabras, presente:1 ausente:0
vocabulario=['fortune','next','programming','money','you']
W = {}
for i,w in enumerate(vocabulario):
    W[i] = {w:mb.Var(name='W'+str(i),values=[0,1])}

for i in range(len(W)):
    print(f'W[{i}]: {W[i]}')

W[0]: {'fortune': <mib_v1_2.Var object at 0x7f8ef5407fd0>}
W[1]: {'next': <mib_v1_2.Var object at 0x7f8ef5407bb0>}
W[2]: {'programming': <mib_v1_2.Var object at 0x7f8ef5407cd0>}
W[3]: {'money': <mib_v1_2.Var object at 0x7f8ef5407b80>}
W[4]: {'you': <mib_v1_2.Var object at 0x7f8ef5407ee0>}


### Descomposición

$P(Spam)$

In [36]:
PSpam=mb.Distrib('P(Spam)',[Spam],{0:.25, 1:.75})
PSpam._print()

('Spam', 0) 0.25
('Spam', 1) 0.75
Printing:  P(Spam)
('Spam', 0):0.25  ('Spam', 1):0.75  



$P(W_i|Spam)$

In [38]:
nif = [0,125,250,0,125]
niv = [375,0,0,750,375]

conteos={}
for i,word,h,s in zip(W,vocabulario,nif,niv):
    var = W[i][word]
    conteos[var.name]={'ham':h,'spam':s}

print(f'Conteos iniciales:\n{conteos}')

ns = 750
nh = 250

def p_ham(var):
    pw_1=(1+conteos[var.name]['ham'])/(var.card+nh)
    pw_0=1.0-pw_1
    return (pw_0,pw_1)

def p_spam(var):
    pw_1=(1+conteos[var.name]['spam'])/(var.card+ns)
    pw_0=1.0-pw_1
    return (pw_0,pw_1)

pw_=[]
for i,word in zip(W,vocabulario):
    pw_.append([p_ham(W[i][word]),p_spam(W[i][word])])

P_W_Spam=[]
for i,word,t in zip(W,vocabulario,pw_):
    print(t)
    P_W_Spam.append(mb.DistribCond(name='P(W'+str(i)+'|Spam)',var=W[i][word],indep=[Spam],tabla=t))

for distrib in P_W_Spam:
    print(distrib.name,'\n',distrib.to_Frame(),'\n')

Conteos iniciales:
{'W0': {'ham': 0, 'spam': 375}, 'W1': {'ham': 125, 'spam': 0}, 'W2': {'ham': 250, 'spam': 0}, 'W3': {'ham': 0, 'spam': 750}, 'W4': {'ham': 125, 'spam': 375}}
[(0.996031746031746, 0.003968253968253968), (0.5, 0.5)]


AttributeError: 'list' object has no attribute 'values'

In [ ]:
spam_value = list(P_W_Spam[0].get_P().keys())
columnas = ['Wi=false','Wi=true']

registros=[]
for d in P_W_Spam:
    tabla=d.get_P().values()
    reg=[]
    for dico in tabla:
        for k in dico.keys():
            reg.append(dico[k])
    registros.append(reg)

# print(spam_value,columnas,registros)

df=pd.DataFrame(registros)
df.columns=pd.MultiIndex.from_product([spam_value,columnas,['P(Wi|Spam)']])
df

(Spam, 0)             (Spam, 1)           
    Wi=false    Wi=true   Wi=false    Wi=true
  P(Wi|Spam) P(Wi|Spam) P(Wi|Spam) P(Wi|Spam)
0   0.996032   0.003968    0.50000    0.50000
1   0.500000   0.500000    0.99867    0.00133
2   0.003968   0.996032    0.99867    0.00133
3   0.996032   0.003968    0.00133    0.99867
4   0.500000   0.500000    0.50000    0.50000

### Distribución conjunta
$P(Spam)\displaystyle \prod_i P(W_i|Spam)$

In [ ]:
PSpamWs=mb.JointDistrib('P(SpamWs)',[Spam]+[d.var for d in P_W_Spam],\
                         [PSpam]+[d for d in P_W_Spam])
PSpamWs._print()

P(W0|Spam) : {('Spam', 0): {('W0', 0): 0.996031746031746, ('W0', 1): 0.003968253968253968}, ('Spam', 1): {('W0', 0): 0.5, ('W0', 1): 0.5}}
P(W1|Spam) : {('Spam', 0): {('W1', 0): 0.5, ('W1', 1): 0.5}, ('Spam', 1): {('W1', 0): 0.9986702127659575, ('W1', 1): 0.0013297872340425532}}
P(W2|Spam) : {('Spam', 0): {('W2', 0): 0.003968253968253954, ('W2', 1): 0.996031746031746}, ('Spam', 1): {('W2', 0): 0.9986702127659575, ('W2', 1): 0.0013297872340425532}}
P(W3|Spam) : {('Spam', 0): {('W3', 0): 0.996031746031746, ('W3', 1): 0.003968253968253968}, ('Spam', 1): {('W3', 0): 0.0013297872340425343, ('W3', 1): 0.9986702127659575}}
P(W4|Spam) : {('Spam', 0): {('W4', 0): 0.5, ('W4', 1): 0.5}, ('Spam', 1): {('W4', 0): 0.5, ('W4', 1): 0.5}}
P(Spam) : {('Spam', 0): 0.25, ('Spam', 1): 0.75}


### Pregunta

In [ ]:
Q_Spam=mb.Question(PSpamWs)

In [ ]:
PSpam_WS=Q_Spam.query([Spam],[d.var for d in P_W_Spam])
df1=PSpam_WS.to_Frame(formateo=True)
df1.reset_index()

computing conditional: P(Spam|W0,W1,W2,W3,W4)


index               (Spam, 0)  \
0   (W0, W1, W2, W3, W4, 0, 0, 0, 0, 0)                0.497351   
1   (W0, W1, W2, W3, W4, 0, 0, 0, 0, 1)                0.497351   
2   (W0, W1, W2, W3, W4, 0, 0, 0, 1, 0)   5.249068925585543e-06   
3   (W0, W1, W2, W3, W4, 0, 0, 0, 1, 1)   5.249068925585543e-06   
4   (W0, W1, W2, W3, W4, 0, 0, 1, 0, 0)                0.999995   
5   (W0, W1, W2, W3, W4, 0, 0, 1, 0, 1)                0.999995   
6   (W0, W1, W2, W3, W4, 0, 0, 1, 1, 0)                0.497351   
7   (W0, W1, W2, W3, W4, 0, 0, 1, 1, 1)                0.497351   
8   (W0, W1, W2, W3, W4, 0, 1, 0, 0, 0)                0.998656   
9   (W0, W1, W2, W3, W4, 0, 1, 0, 0, 1)                0.998656   
10  (W0, W1, W2, W3, W4, 0, 1, 0, 1, 0)                0.003927   
11  (W0, W1, W2, W3, W4, 0, 1, 0, 1, 1)                0.003927   
12  (W0, W1, W2, W3, W4, 0, 1, 1, 0, 0)                1.000000   
13  (W0, W1, W2, W3, W4, 0, 1, 1, 0, 1)                1.000000   
14  (W0, W1, W2, W3, W4, 0, 1, 1, 1, 0)                0.998656   
15  (W0, W1, W2, W3, W4, 0, 1, 1, 1, 1)                0.998656   
16  (W0, W1, W2, W3, W4, 1, 0, 0, 0, 0)                0.003927   
17  (W0, W1, W2, W3, W4, 1, 0, 0, 0, 1)                0.003927   
18  (W0, W1, W2, W3, W4, 1, 0, 0, 1, 0)   2.091273453658085e-08   
19  (W0, W1, W2, W3, W4, 1, 0, 0, 1, 1)   2.091273453658085e-08   
20  (W0, W1, W2, W3, W4, 1, 0, 1, 0, 0)                0.998656   
21  (W0, W1, W2, W3, W4, 1, 0, 1, 0, 1)                0.998656   
22  (W0, W1, W2, W3, W4, 1, 0, 1, 1, 0)                0.003927   
23  (W0, W1, W2, W3, W4, 1, 0, 1, 1, 1)                0.003927   
24  (W0, W1, W2, W3, W4, 1, 1, 0, 0, 0)                0.747506   
25  (W0, W1, W2, W3, W4, 1, 1, 0, 0, 1)                0.747506   
26  (W0, W1, W2, W3, W4, 1, 1, 0, 1, 0)  1.5705217307691924e-05   
27  (W0, W1, W2, W3, W4, 1, 1, 0, 1, 1)  1.5705217307691924e-05   
28  (W0, W1, W2, W3, W4, 1, 1, 1, 0, 0)                0.999998   
29  (W0, W1, W2, W3, W4, 1, 1, 1, 0, 1)                0.999998   
30  (W0, W1, W2, W3, W4, 1, 1, 1, 1, 0)                0.747506   
31  (W0, W1, W2, W3, W4, 1, 1, 1, 1, 1)                0.747506   

                 (Spam, 1)  
0                 0.502649  
1                 0.502649  
2                 0.999995  
3                 0.999995  
4    5.361493692197752e-06  
5    5.361493692197752e-06  
6                 0.502649  
7                 0.502649  
8                 0.001344  
9                 0.001344  
10                0.996073  
11                0.996073  
12   7.139177629413613e-09  
13   7.139177629413613e-09  
14                0.001344  
15                0.001344  
16                0.996073  
17                0.996073  
18                1.000000  
19                1.000000  
20                0.001344  
21                0.001344  
22                0.996073  
23                0.996073  
24                0.252494  
25                0.252494  
26                0.999984  
27                0.999984  
28  1.7919303867554844e-06  
29  1.7919303867554844e-06  
30                0.252494  
31                0.252494

In [ ]:
PSpam_WS=Q_Spam.query([Spam],[d.var for d in P_W_Spam])
df2=PSpam_WS.to_Frame(formateo=True)
df2

computing conditional: P(Spam|W0,W1,W2,W3,W4)


(Spam, 0)  \
(W0, W1, W2, W3, W4, 0, 0, 0, 0, 0)                0.497351   
(W0, W1, W2, W3, W4, 0, 0, 0, 0, 1)                0.497351   
(W0, W1, W2, W3, W4, 0, 0, 0, 1, 0)   5.249068925585543e-06   
(W0, W1, W2, W3, W4, 0, 0, 0, 1, 1)   5.249068925585543e-06   
(W0, W1, W2, W3, W4, 0, 0, 1, 0, 0)                0.999995   
(W0, W1, W2, W3, W4, 0, 0, 1, 0, 1)                0.999995   
(W0, W1, W2, W3, W4, 0, 0, 1, 1, 0)                0.497351   
(W0, W1, W2, W3, W4, 0, 0, 1, 1, 1)                0.497351   
(W0, W1, W2, W3, W4, 0, 1, 0, 0, 0)                0.998656   
(W0, W1, W2, W3, W4, 0, 1, 0, 0, 1)                0.998656   
(W0, W1, W2, W3, W4, 0, 1, 0, 1, 0)                0.003927   
(W0, W1, W2, W3, W4, 0, 1, 0, 1, 1)                0.003927   
(W0, W1, W2, W3, W4, 0, 1, 1, 0, 0)                1.000000   
(W0, W1, W2, W3, W4, 0, 1, 1, 0, 1)                1.000000   
(W0, W1, W2, W3, W4, 0, 1, 1, 1, 0)                0.998656   
(W0, W1, W2, W3, W4, 0, 1, 1, 1, 1)                0.998656   
(W0, W1, W2, W3, W4, 1, 0, 0, 0, 0)                0.003927   
(W0, W1, W2, W3, W4, 1, 0, 0, 0, 1)                0.003927   
(W0, W1, W2, W3, W4, 1, 0, 0, 1, 0)   2.091273453658085e-08   
(W0, W1, W2, W3, W4, 1, 0, 0, 1, 1)   2.091273453658085e-08   
(W0, W1, W2, W3, W4, 1, 0, 1, 0, 0)                0.998656   
(W0, W1, W2, W3, W4, 1, 0, 1, 0, 1)                0.998656   
(W0, W1, W2, W3, W4, 1, 0, 1, 1, 0)                0.003927   
(W0, W1, W2, W3, W4, 1, 0, 1, 1, 1)                0.003927   
(W0, W1, W2, W3, W4, 1, 1, 0, 0, 0)                0.747506   
(W0, W1, W2, W3, W4, 1, 1, 0, 0, 1)                0.747506   
(W0, W1, W2, W3, W4, 1, 1, 0, 1, 0)  1.5705217307691924e-05   
(W0, W1, W2, W3, W4, 1, 1, 0, 1, 1)  1.5705217307691924e-05   
(W0, W1, W2, W3, W4, 1, 1, 1, 0, 0)                0.999998   
(W0, W1, W2, W3, W4, 1, 1, 1, 0, 1)                0.999998   
(W0, W1, W2, W3, W4, 1, 1, 1, 1, 0)                0.747506   
(W0, W1, W2, W3, W4, 1, 1, 1, 1, 1)                0.747506   

                                                  (Spam, 1)  
(W0, W1, W2, W3, W4, 0, 0, 0, 0, 0)                0.502649  
(W0, W1, W2, W3, W4, 0, 0, 0, 0, 1)                0.502649  
(W0, W1, W2, W3, W4, 0, 0, 0, 1, 0)                0.999995  
(W0, W1, W2, W3, W4, 0, 0, 0, 1, 1)                0.999995  
(W0, W1, W2, W3, W4, 0, 0, 1, 0, 0)   5.361493692197752e-06  
(W0, W1, W2, W3, W4, 0, 0, 1, 0, 1)   5.361493692197752e-06  
(W0, W1, W2, W3, W4, 0, 0, 1, 1, 0)                0.502649  
(W0, W1, W2, W3, W4, 0, 0, 1, 1, 1)                0.502649  
(W0, W1, W2, W3, W4, 0, 1, 0, 0, 0)                0.001344  
(W0, W1, W2, W3, W4, 0, 1, 0, 0, 1)                0.001344  
(W0, W1, W2, W3, W4, 0, 1, 0, 1, 0)                0.996073  
(W0, W1, W2, W3, W4, 0, 1, 0, 1, 1)                0.996073  
(W0, W1, W2, W3, W4, 0, 1, 1, 0, 0)   7.139177629413613e-09  
(W0, W1, W2, W3, W4, 0, 1, 1, 0, 1)   7.139177629413613e-09  
(W0, W1, W2, W3, W4, 0, 1, 1, 1, 0)                0.001344  
(W0, W1, W2, W3, W4, 0, 1, 1, 1, 1)                0.001344  
(W0, W1, W2, W3, W4, 1, 0, 0, 0, 0)                0.996073  
(W0, W1, W2, W3, W4, 1, 0, 0, 0, 1)                0.996073  
(W0, W1, W2, W3, W4, 1, 0, 0, 1, 0)                1.000000  
(W0, W1, W2, W3, W4, 1, 0, 0, 1, 1)                1.000000  
(W0, W1, W2, W3, W4, 1, 0, 1, 0, 0)                0.001344  
(W0, W1, W2, W3, W4, 1, 0, 1, 0, 1)                0.001344  
(W0, W1, W2, W3, W4, 1, 0, 1, 1, 0)                0.996073  
(W0, W1, W2, W3, W4, 1, 0, 1, 1, 1)                0.996073  
(W0, W1, W2, W3, W4, 1, 1, 0, 0, 0)                0.252494  
(W0, W1, W2, W3, W4, 1, 1, 0, 0, 1)                0.252494  
(W0, W1, W2, W3, W4, 1, 1, 0, 1, 0)                0.999984  
(W0, W1, W2, W3, W4, 1, 1, 0, 1, 1)                0.999984  
(W0, W1, W2, W3, W4, 1, 1, 1, 0, 0)  1.7919303867554844e-06  
(W0, W1, W2, W3, W4, 1, 1, 1, 0, 1)  1.7919303867554844e-06  
(W0, W1, W2, W3, W4, 1, 1, 1, 1, 0)             

## TAREA PARA ENTREGAR (INDIVIDUAL)

#### Planteamiento

Una compañía sabe que la contratación profesional de un candidato se efectúa luego de una entrevista, que se lleva a cabo en función de las calificaciones reportadas en el certificado del candidato y si éste tiene experiencia laboral o no. Generalmente, sólo se consideran los candidatos con calificaciones sobresalientes o regulares, y la entrevista arroja típicamente tres tipos de apreciación.

La compañía ha elaborado el siguiente modelo de contratación:

$\;\;\;\;\;\;\;\;\;\;\;\;$<img src="./MODELO_TAREA.jpg" width=300/>

La compañía ha recolectado datos históricos de los últimos 5 años sobre la evaluación de 500 candidatos y sabe que la proporción de candidatos con calificaciones regulares es del 30%, mientras que los candidatos con experiencia laboral representan el 60%. En la mejor condición, con calificaciones sobresalientes y experiencia laboral, los candidatos obtienen la más alta apreciación en la entrevista en un 80%, y la peor apreciación en un 2% de los casos. Estos porcentajes cambian respectivamente a 30% y 10% cuando las calificaciones no son las mejores, pero sí hay experiencia, y ambas son del 30% con calificaciones sobresalientes pero sin experiencia. En el peor caso, con calificaciones regulares y sin experiencia, los candidatos obtienen la mejor apreciación en la entrevista en un 10% y la peor en un 70%. Por último, se sabe que la tasa de candidatos rechazados es de un 10% cuando obtienen una apreciación favorable, un 60% cuando obtienen una apreciación regular, y un 99% cuando obtienen una apreciación desfavorable en la entrevista.

#### Ojetivos
La compañía desea saber:
1. Cuál es su tasa de contratación (i.e. cuál es la probabilidad de ser contratado).
2. Cuantos candidatos ha contratado la compañía en los últimos 5 años.
3. Cuál es la tasa de apreciaciones favorables en la compañía.
4. Cuál es la tasa de apreciaciones regulares dado que se contrata a alguien. 
5. Cuantos de los candidatos contratados obtuvieron una apreciación favorable.
6. Cuantos de los candidatos contratados obtuvieron una apreciación regular.
7. Cómo se distribuye la contratación de candidatos en función de sus calificaciones.
8. Cómo se distribuye la contratación de candidatos en función de su experiencia.
9. Cuál es la probabilidad de que alguien con experiencia laboral tenga calificaciones sobresalientes.
10. Cuál es la distribución conjunta del personal contratado y la apreciación de la entrevista. 
11. ¿Diría usted que la compañía está haciendo bien su trabajo de contratación? Explique. 